In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import spacy
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
file=r"C:\Users\jayen\Text-sentiment-analysis-general-purpose\artifacts\Forsentiment2.csv"
#n=100000
df=pd.read_csv(file)
df


,Unnamed: 0,text,label
0,0,"According to Gran , the company has no plans t...",neutral
1,1,Technopolis plans to develop in stages an area...,neutral
2,2,The international electronic industry company ...,negative
3,3,With the new production plant the company woul...,positive
4,4,According to the company 's updated strategy f...,positive
...,...,...,...
512672,531249,"Man, I loved this movie! This really takes me ...",positive
512673,531250,Recovery is an incredibly moving piece of work...,positive
512674,531251,"You can take the crook out of the joint, but i...",positive
512675,531252,FUTZ is the only show preserved from the exper...,positive


In [11]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531254 entries, 0 to 531253
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    512684 non-null  object
 1   label   512800 non-null  object
dtypes: object(2)
memory usage: 8.1+ MB


In [33]:
class_label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
# Map class names to numerical labels
df['label'] = df['label'].replace(class_label_mapping)

In [29]:
l=LabelEncoder()
df['labels']=l.fit_transform(df['label'])

In [34]:
df

,Unnamed: 0,text,label
0,0,"According to Gran , the company has no plans t...",0
1,1,Technopolis plans to develop in stages an area...,0
2,2,The international electronic industry company ...,2
3,3,With the new production plant the company woul...,1
4,4,According to the company 's updated strategy f...,1
...,...,...,...
512672,531249,"Man, I loved this movie! This really takes me ...",1
512673,531250,Recovery is an incredibly moving piece of work...,1
512674,531251,"You can take the crook out of the joint, but i...",1
512675,531252,FUTZ is the only show preserved from the exper...,1


In [21]:
df

,Unnamed: 0,text,label
0,0,"According to Gran , the company has no plans t...",neutral
1,1,Technopolis plans to develop in stages an area...,neutral
2,2,The international electronic industry company ...,negative
3,3,With the new production plant the company woul...,positive
4,4,According to the company 's updated strategy f...,positive
...,...,...,...
512672,531249,"Man, I loved this movie! This really takes me ...",positive
512673,531250,Recovery is an incredibly moving piece of work...,positive
512674,531251,"You can take the crook out of the joint, but i...",positive
512675,531252,FUTZ is the only show preserved from the exper...,positive


In [12]:
df['label']=df['label'].replace('normal','neutral')

In [35]:
df['label'].value_counts()

label
1    323305
2    103495
0     85877
Name: count, dtype: int64

In [36]:
#there are some missing values in label and in text too
df.isnull().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [15]:
df=df.dropna()



In [18]:
df['label'].value_counts()

label
positive    323305
negative    103495
neutral      85877
Name: count, dtype: int64

In [37]:
df.to_csv(r'C:\Users\jayen\Text-sentiment-analysis-general-purpose\artifacts\Forsentiment3.csv')

In [6]:
#NOW ILL DO SOME PREPROCESSING STEPS FOR THAT ILL CREATE A FUNCTION
def preprocessing(q):
 
  q=str(q).lower().strip()#strip gives a string in return if u use split it will give a list of words
  #The strip() method in Python is used to remove leading and trailing characters (whitespace characters by default) from a string before " Hello, world! " after"Hello, world!"
   # Replace certain special characters with their string equivalents
  q = q.replace('%', 'percent')
  q = q.replace('$', 'dollar ')
  q = q.replace('₹', 'rupee ')
  q = q.replace('€', 'euro ')
  q = q.replace('@', 'at')
  q=q.replace('&','and')

  q = re.sub(r'([0-9]+)000000000', r'\1b', re.sub(r'([0-9]+)000000', r'\1m', re.sub(r'([0-9]+)000', r'\1k', q)))#Input: "1500000000 Output: "1.5b" it will do this 



    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
  q_decontracted=[]

  q_decontracted = [contractions[i] if i in contractions else i for i in q.split()]


  #now i join this word into q
  q=' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")
  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()
  # Remove punctuations like ,.""[]
  q =re.sub(r'[^\w\s{}\/<>\'"\-_=+()*~:;]', ' ', q)

  q = word_tokenize(q)  # Convert to lowercase

  # Remove stopwords and non-alphabetic characters, perform lemmatization
  q = [
        lemmatizer.lemmatize(word) for word in q
        if word not in stop_words
    ]



  return ' '.join(q)#ALWAYS REMEMBER IN ORDER TO USE WORD2VEC UR SENTENSES SHOULD BE IN TOKKENS OR WORDS BUT IN THIS CASE WE HAVE TO TRAIN OUR MODEL WE HAVE TO TOOKENIZE
  #THE WORDS SO WE USE JOIN THEN AFTER WE TOKKENIZE IT WONT BE ABLE TO CONCATINATE LIST OF STRINGS IMPORTANT. contain lists instead of strings.





In [7]:
df['preprocessed']=df['text'].apply(preprocessing)

C:\Users\jayen\AppData\Local\Temp\ipykernel_5600\495576266.py:151: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  q = BeautifulSoup(q)


In [24]:
#TRIED THE SPACY MODEL BUT ITS TAKING MORE TIME TO PROCEES EACH SENETENSES FOR PREPROCESSING ALWAYS USE NLTK  U CAN SEE THE RUNTIME

nlp = spacy.load("en_core_web_sm")
doc = nlp("Finnish Okmetic that manufactures and processes silicon wafers for the semiconductor and sensor industries and Norwegian solar wafer company NorSun have signed a contract under which Okmetic will supply NorSun mono silicon crystals for use in solar cell manufacturing .,positive"

)

  # Remove stop words and perform lemmatization
a = [token.lemma_ for token in doc if not token.is_stop]
print(a)

['finnish', 'Okmetic', 'manufacture', 'process', 'silicon', 'wafer', 'semiconductor', 'sensor', 'industry', 'norwegian', 'solar', 'wafer', 'company', 'NorSun', 'sign', 'contract', 'Okmetic', 'supply', 'NorSun', 'mono', 'silicon', 'crystal', 'use', 'solar', 'cell', 'manufacturing', '.,positive']


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['Finnish', 'real', 'estate', 'investor', 'Sponda', 'Plc', 'said', 'Wednesday', '12', 'March', 'signed', 'agreement', 'Danske', 'Bank', 'A-S', ',', 'Helsinki', 'Branch', '7-year', 'EUR150m', 'credit', 'facility', 'Ilmarinen', 'Mutual', 'Pension', 'Insurance', 'Company', '7-year', 'EUR50m', 'credit', 'facility', '.']


In [25]:
q = word_tokenize("Finnish Okmetic that manufactures and processes silicon wafers for the semiconductor and sensor industries and Norwegian solar wafer company NorSun have signed a contract under which Okmetic will supply NorSun mono silicon crystals for use in solar cell manufacturing .,positive")  # Convert to lowercase

# Remove stopwords and non-alphabetic characters, perform lemmatization
q = [
    lemmatizer.lemmatize(word) for word in q
    if word not in stop_words
]

print(q)

['Finnish', 'Okmetic', 'manufacture', 'process', 'silicon', 'wafer', 'semiconductor', 'sensor', 'industry', 'Norwegian', 'solar', 'wafer', 'company', 'NorSun', 'signed', 'contract', 'Okmetic', 'supply', 'NorSun', 'mono', 'silicon', 'crystal', 'use', 'solar', 'cell', 'manufacturing', '.', ',', 'positive']


In [43]:
df

,text,label,preprocessed
0,"According to Gran , the company has no plans t...",neutral,according gran company plan move production ru...
1,Technopolis plans to develop in stages an area...,neutral,technopolis plan develop stage area le 100 000...
2,The international electronic industry company ...,negative,international electronic industry company elco...
3,With the new production plant the company woul...,positive,new production plant company would increase ca...
4,According to the company 's updated strategy f...,positive,according company 's updated strategy year 200...
...,...,...,...
531249,"Man, I loved this movie! This really takes me ...",positive,man loved movie really take back kid day teach...
531250,Recovery is an incredibly moving piece of work...,positive,recovery incredibly moving piece work handling...
531251,"You can take the crook out of the joint, but i...",positive,take crook joint seems exceedingly difficult t...
531252,FUTZ is the only show preserved from the exper...,positive,futz show preserved experimental theatre movem...


In [8]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed'])
sequences = tokenizer.texts_to_sequences(df['preprocessed'])

In [10]:
max_length = 200  # Define your maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length)#THIS WILL BE MY ACTUAL DATA THAT'LL TRAIN TEST AND SPLIT 

In [12]:
padded_sequences[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
word_index = tokenizer.word_index#THESE ARE MY UNIQUE WORDS WITH ITS UNIQUE INDEX
print(word_index)

In [ ]:
num_words = len(word_index)+1#THIS WILL GET THE TOTAL NUMBER OF UNIQUE WORDS LENTTH MEANS VOCABULARY 

In [13]:
## Creating model
from tensorflow.keras.layers import Dropout
embedding_vector_features=300 ##features representation each word wil get convert into a vector of 300
model=Sequential()
model.add(Embedding(num_words,embedding_vector_features,input_length=max_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(3,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          9235800   
                                                                 
 dropout (Dropout)           (None, 200, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 9396503 (35.84 MB)
Trainable params: 9396503 (35.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
class_label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
# Map class names to numerical labels
df['label'] = df['label'].replace(class_label_mapping)

In [15]:
df['label'].value_counts()

label
1    61693
2    10605
0     9285
Name: count, dtype: int64

In [16]:
import numpy as np
X_final=np.array(padded_sequences)
y_final=np.array(df['label'])

In [88]:
X_final[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [19]:
X_final.shape,y_final.shape

((281434, 200), (281434,))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=3, mode='max', verbose=1, baseline=0.95)

In [19]:
model.fit(X_final,y_final, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
2040/2040 [==============================] - 643s 312ms/step - loss: 0.3566 - accuracy: 0.8702 - val_loss: 0.2543 - val_accuracy: 0.9169
Epoch 2/10
2040/2040 [==============================] - 615s 302ms/step - loss: 0.2276 - accuracy: 0.9213 - val_loss: 0.2506 - val_accuracy: 0.9154
Epoch 3/10
2040/2040 [==============================] - 644s 316ms/step - loss: 0.1749 - accuracy: 0.9390 - val_loss: 0.2606 - val_accuracy: 0.9132
Epoch 4/10
2040/2040 [==============================] - 681s 334ms/step - loss: 0.1440 - accuracy: 0.9496 - val_loss: 0.2808 - val_accuracy: 0.9094
Epoch 5/10
2040/2040 [==============================] - 680s 333ms/step - loss: 0.1269 - accuracy: 0.9552 - val_loss: 0.2882 - val_accuracy: 0.9136
Epoch 6/10
2040/2040 [==============================] - 684s 335ms/step - loss: 0.1140 - accuracy: 0.9597 - val_loss: 0.2964 - val_accuracy: 0.9114
Epoch 7/10
2040/2040 [==============================] - 780s 382ms/step - loss: 0.1055 - accuracy: 0.9630 - val_

In [20]:
import os
artifacts_folder = "artifacts"
os.makedirs(artifacts_folder, exist_ok=True)
# Save the trained model to the "artifacts" folder
model_filename = os.path.join(artifacts_folder, 'chat_model')
model.save(model_filename)

INFO:tensorflow:Assets written to: artifacts\chat_model\assets


INFO:tensorflow:Assets written to: artifacts\chat_model\assets


: 